In [256]:
import pandas as pd
import numpy as np

Table to retrace emails for when the new survey must be sent out

In [ ]:
analysis_df = pd.read_csv("dss_data22nov.csv")
contact_df = analysis_df[["subject", "email"]].copy()
contact_df = contact_df.dropna(subset=["email"])
contact_group2 = analysis_df[analysis_df["group"] == 2][["subject", "email"]].dropna(subset=["email"])
contact_group2

,subject,email
0,anonymous,pieter.drabbe@hotmail.com
3,S105,kittytail@hotmail.co.uk
4,S106,harryhall3698@gmail.com
5,S107,l.j.vanvoorden@tilburguniversity.edu
13,S128,r.s.kramer_1@tilburguniversity.edu
14,S129,j.m.spuesens@tilburguniversity.edu
16,S137,Alettakramer@gmail.com
17,S139,R.kramer@genexis.eu
18,S140,Kramer.marijke@kpnmail.nl
19,S148,roosvanvoorden2004@gmail.com


Load the data

In [258]:
ratings_path = "dss_data22nov.csv"   # survey data
songs_path   = "candDSS.csv"         # song features

In [259]:
df_ratings = pd.read_csv(ratings_path)
df_songs   = pd.read_csv(songs_path)

We keep only group 2 participants

In [260]:
df_g2 = df_ratings[df_ratings["group"] == 2].copy()

Sorting the columns

In [261]:
rating_cols = [
    c for c in df_g2.columns
    if c.startswith("s") and c.endswith("r") and c[1:-1].isdigit()
]

fam_cols = [
    c for c in df_g2.columns
    if c.startswith("f") and c[1:].isdigit()
]

print("Number of participants", len(df_g2))
print("Number of rating columns:", len(rating_cols))

Number of participants 23
Number of rating columns: 60


Clean data and adjust format

In [262]:
for col in rating_cols:
    df_g2[col] = pd.to_numeric(df_g2[col], errors="coerce")

for col in fam_cols:
    df_g2[col] = df_g2[col].replace("new", "new") 

Extract Song IDs

In [263]:
long_ratings = df_g2.melt(
    id_vars=["subject"],          
    value_vars=rating_cols,  
    var_name="rating_col",
    value_name="rating"
)

# Drop NAs
long_ratings = long_ratings.dropna(subset=["rating"])

# Extract song ID from column name
long_ratings["song_id"] = (
    long_ratings["rating_col"]
    .str.extract(r"s(\d+)r")
    .astype(int)
)

In [264]:
df_long = long_ratings

Make a new dataframe with song features

In [265]:
song_features = df_songs[[
    "id", "genre_allmusic", "trackname",
    "danceability", "valence", "energy",
    "acousticness", "instrumentalness",
    "speechiness", "liveness",
    "tempo", "loudness", "density"
]].rename(columns={"id": "song_id"})

In [266]:
analysis_df = df_long.merge(
    song_features,
    on="song_id",
    how="left"
)

Sort the dataframe by user

In [267]:
analysis_df = analysis_df.sort_values(by="subject").reset_index(drop=True)

In [268]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

analysis_df


,subject,rating_col,rating,song_id,genre_allmusic,trackname,danceability,valence,energy,acousticness,instrumentalness,speechiness,liveness,tempo,loudness,density
0,S105,s2r,3.0,2,pop-rock,Beast Of Burden - Remastered,0.783,0.8850,0.8780,0.390000,0.000778,0.0308,0.0426,100.626,-3.864,3.247493
1,S105,s1r,5.0,1,pop-rock,"More Than A Woman - From ""Saturday Night Fever...",0.601,0.6730,0.7030,0.175000,0.000084,0.0379,0.5710,106.164,-6.240,19.365283
2,S105,s30r,3.0,30,country,I'm Alright,0.676,0.5910,0.7330,0.191000,0.000000,0.0294,0.1920,98.234,-7.745,10.100652
3,S105,s21r,3.0,21,country,Blue Eyes Crying In the Rain,0.626,0.4180,0.0400,0.928000,0.001780,0.0405,0.1100,117.136,-19.994,2.241434
4,S105,s57r,5.0,57,rnb,A Teenager In Love,0.555,0.6190,0.3770,0.671000,0.000000,0.0295,0.1280,78.986,-9.123,10.099300
5,S105,s50r,1.0,50,rap,You Know How We Do It - Remastered,0.747,0.6010,0.5200,0.097700,0.053700,0.0357,0.1420,92.869,-10.523,25.360962
6,S105,s20r,5.0,20,classical,Epilogue,0.190,0.0786,0.0739,0.975000,0.110000,0.0333,0.0725,99.994,-21.400,149.172613
7,S105,s6r,4.0,6,pop-rock,She Sells Sanctuary,0.550,0.4190,0.7650,0.000005,0.873000,0.0287,0.1320,139.460,-8.684,144.212026
8,S105,s55r,3.0,55,rnb,Have You Ever,0.698,0.2750,0.5330,0.542000,0.000000,0.0437,0.3330,134.001,-6.246,0.446351
9,S105,s26r,4.0,26,country,Aw Naw,0.652,0.4910,0.8590,0.063900,0.000000,0.0358,0.0777,109.971,-3.746,2.987505


Make user profiles

We add an extra column that states a user's opinion on the song based on their rating

In [269]:
analysis_df["like_label"] = np.where(
    analysis_df["rating"] >= 4, "like",
    np.where(analysis_df["rating"] <= 2, "dislike", "neutral")
)

analysis_df["like_label"].value_counts()


like_label
dislike    159
neutral     97
like        89
Name: count, dtype: int64

In [270]:
analysis_df.head()

,subject,rating_col,rating,song_id,genre_allmusic,trackname,danceability,valence,energy,acousticness,instrumentalness,speechiness,liveness,tempo,loudness,density,like_label
0,S105,s2r,3.0,2,pop-rock,Beast Of Burden - Remastered,0.783,0.885,0.878,0.390,0.000778,0.0308,0.0426,100.626,-3.864,3.247493,neutral
1,S105,s1r,5.0,1,pop-rock,"More Than A Woman - From ""Saturday Night Fever...",0.601,0.673,0.703,0.175,0.000084,0.0379,0.5710,106.164,-6.240,19.365283,like
2,S105,s30r,3.0,30,country,I'm Alright,0.676,0.591,0.733,0.191,0.000000,0.0294,0.1920,98.234,-7.745,10.100652,neutral
3,S105,s21r,3.0,21,country,Blue Eyes Crying In the Rain,0.626,0.418,0.040,0.928,0.001780,0.0405,0.1100,117.136,-19.994,2.241434,neutral
4,S105,s57r,5.0,57,rnb,A Teenager In Love,0.555,0.619,0.377,0.671,0.000000,0.0295,0.1280,78.986,-9.123,10.099300,like


We use the new like column to set up the user profiles

In [271]:
feature_cols = [
    "danceability",
    "valence",
    "energy",
    "acousticness",
    "instrumentalness",
    "speechiness",
    "liveness",
    "tempo",
    "loudness",
    "density"
]

In [272]:
liked = analysis_df[analysis_df["like_label"] == "like"]

user_profile_like = (
    liked.groupby("subject")[feature_cols]
         .mean()
         .reset_index()
         .rename(columns=lambda c: c + "_like" if c in feature_cols else c)
)


In [273]:
user_profile_like.head()


,subject,danceability_like,valence_like,energy_like,acousticness_like,instrumentalness_like,speechiness_like,liveness_like,tempo_like,loudness_like,density_like
0,S105,0.509600,0.456120,0.555580,0.376981,0.196617,0.033040,0.196240,106.915,-9.838600,65.167345
1,S106,0.589000,0.457700,0.524267,0.386165,0.298525,0.047678,0.139167,111.059,-11.940556,135.614987
2,S107,0.589333,0.614967,0.591667,0.486667,0.000328,0.035467,0.230200,105.064,-9.755000,7.867858
3,S128,0.542250,0.395750,0.624500,0.046751,0.438521,0.033100,0.120825,114.940,-8.826250,43.017828
4,S129,0.703500,0.681000,0.613500,0.247350,0.026850,0.121850,0.176500,91.074,-8.140500,14.012684


The user profiles now show the average scores users had for each feature in songs they liked

In [274]:
analysis_df = analysis_df.merge(user_profile_like, on="subject", how="left")
analysis_df.head()

,subject,rating_col,rating,song_id,genre_allmusic,trackname,danceability,valence,energy,acousticness,instrumentalness,speechiness,liveness,tempo,loudness,density,like_label,danceability_like,valence_like,energy_like,acousticness_like,instrumentalness_like,speechiness_like,liveness_like,tempo_like,loudness_like,density_like
0,S105,s2r,3.0,2,pop-rock,Beast Of Burden - Remastered,0.783,0.885,0.878,0.390,0.000778,0.0308,0.0426,100.626,-3.864,3.247493,neutral,0.5096,0.45612,0.55558,0.376981,0.196617,0.03304,0.19624,106.915,-9.8386,65.167345
1,S105,s1r,5.0,1,pop-rock,"More Than A Woman - From ""Saturday Night Fever...",0.601,0.673,0.703,0.175,0.000084,0.0379,0.5710,106.164,-6.240,19.365283,like,0.5096,0.45612,0.55558,0.376981,0.196617,0.03304,0.19624,106.915,-9.8386,65.167345
2,S105,s30r,3.0,30,country,I'm Alright,0.676,0.591,0.733,0.191,0.000000,0.0294,0.1920,98.234,-7.745,10.100652,neutral,0.5096,0.45612,0.55558,0.376981,0.196617,0.03304,0.19624,106.915,-9.8386,65.167345
3,S105,s21r,3.0,21,country,Blue Eyes Crying In the Rain,0.626,0.418,0.040,0.928,0.001780,0.0405,0.1100,117.136,-19.994,2.241434,neutral,0.5096,0.45612,0.55558,0.376981,0.196617,0.03304,0.19624,106.915,-9.8386,65.167345
4,S105,s57r,5.0,57,rnb,A Teenager In Love,0.555,0.619,0.377,0.671,0.000000,0.0295,0.1280,78.986,-9.123,10.099300,like,0.5096,0.45612,0.55558,0.376981,0.196617,0.03304,0.19624,106.915,-9.8386,65.167345


In [275]:
top_genre_per_user = (
    analysis_df[analysis_df["like_label"] == "like"]
      .groupby("subject")["genre_allmusic"]
      .agg(lambda x: x.mode().iloc[0])
      .reset_index()
      .rename(columns={"genre_allmusic": "preferred_genre"})
)

analysis_df = analysis_df.merge(top_genre_per_user, on="subject", how="left")

In [276]:
analysis_df.to_csv("cleaned_musicdata.csv", index=False)


Now it's time to generate recommendations. We decided to generate 8 recommendations per user. 2 of their preferred genre with similar characteristics to what they like. 2 of their preferred genre with characteristics farthest from what they like. 2 of genres that are not their preferred genre with similar characteristics to what they like. And 2 of their non-preferred genres with characteristics farthest from what they like. 

The distance comparison was made from a combination of danceability, valence, and energy, as we found these three characteristics to be important ones in assessing how much someone would like a song.

In [286]:
N = 2  # how many similar songs per subject

feat_cols = ["danceability", "valence", "energy"]
like_cols = ["danceability_like", "valence_like", "energy_like"]

We loop over all users, generating these sets of 2 recommendations

In [278]:
rows = []
for subj in analysis_df["subject"].unique():
    # Preferred genre for this subject
    preferred_genre = analysis_df.loc[
        analysis_df["subject"] == subj, "preferred_genre"
    ].iloc[0]
    
    # Subject's like profile
    user_pref = analysis_df.loc[
        analysis_df["subject"] == subj, like_cols
    ].iloc[0]
    
    # Songs in that genre
    songs_same_genre = df_songs[df_songs["genre_allmusic"] == preferred_genre].copy()
    
    # Remove songs the subject already rated
    rated_songs = set(
        analysis_df.loc[analysis_df["subject"] == subj, "song_id"].tolist()
    )
    songs_same_genre = songs_same_genre[
        ~songs_same_genre["id"].isin(rated_songs)
    ].copy()
    
    if songs_same_genre.empty:
        continue  # nothing to recommend for this subject
    
    # Compute distance
    diffs = songs_same_genre[feat_cols].values - user_pref.values
    songs_same_genre["distance"] = np.linalg.norm(diffs, axis=1)
    
    # Take N most similar songs
    most_similar = songs_same_genre.sort_values("distance").head(N)
    
    # Store only the columns you want
    for _, row in most_similar.iterrows():
        rows.append({
            "subject": subj,
            "genre_allmusic": row["genre_allmusic"],
            "trackname": row["trackname"],
            "bucket": "same_genre_similar"
        })

# Final dataframe with only the requested columns
recs_same_genre_similar = pd.DataFrame(rows)


In [279]:
rows = []
for subj in analysis_df["subject"].unique():
    
    # Subject's preferred genre
    preferred_genre = analysis_df.loc[
        analysis_df["subject"] == subj, "preferred_genre"
    ].iloc[0]
    
    # Subject's like profile
    user_pref = analysis_df.loc[
        analysis_df["subject"] == subj, like_cols
    ].iloc[0]
    
    # Songs not in preferred genre
    songs_diff_genre = df_songs[df_songs["genre_allmusic"] != preferred_genre].copy()
    
    # Remove songs already rated
    rated_songs = set(
        analysis_df.loc[analysis_df["subject"] == subj, "song_id"].tolist()
    )
    songs_diff_genre = songs_diff_genre[
        ~songs_diff_genre["id"].isin(rated_songs)
    ].copy()
    
    if songs_diff_genre.empty:
        continue
    
    # Compute distance to user’s preference
    diffs = songs_diff_genre[feat_cols].values - user_pref.values
    songs_diff_genre["distance"] = np.linalg.norm(diffs, axis=1)
    
    # Pick N most similar (smallest distance)
    most_similar = songs_diff_genre.sort_values("distance").head(N)
    
    # Keep only the required columns
    for _, row in most_similar.iterrows():
        rows.append({
            "subject": subj,
            "genre_allmusic": row["genre_allmusic"],
            "trackname": row["trackname"],
            "bucket": "diff_genre_similar"
        })

# Final dataframe
recs_diff_genre_similar = pd.DataFrame(rows)

In [280]:
rows = []
for subj in analysis_df["subject"].unique():
    
    # Subject's preferred genre
    preferred_genre = analysis_df.loc[
        analysis_df["subject"] == subj, "preferred_genre"
    ].iloc[0]
    
    # Subject's like profile
    user_pref = analysis_df.loc[
        analysis_df["subject"] == subj, like_cols
    ].iloc[0]
    
    # Songs in the preferred genre
    songs_same_genre = df_songs[df_songs["genre_allmusic"] == preferred_genre].copy()
    
    # Remove songs already rated
    rated_songs = set(
        analysis_df.loc[analysis_df["subject"] == subj, "song_id"].tolist()
    )
    songs_same_genre = songs_same_genre[
        ~songs_same_genre["id"].isin(rated_songs)
    ].copy()
    
    if songs_same_genre.empty:
        continue
    
    # Compute distance to user’s preference
    diffs = songs_same_genre[feat_cols].values - user_pref.values
    songs_same_genre["distance"] = np.linalg.norm(diffs, axis=1)
    
    # Pick N least similar (largest distance)
    most_dissimilar = songs_same_genre.sort_values("distance", ascending=False).head(N)
    
    # Keep only the required columns
    for _, row in most_dissimilar.iterrows():
        rows.append({
            "subject": subj,
            "genre_allmusic": row["genre_allmusic"],
            "trackname": row["trackname"],
            "bucket": "same_genre_dissimilar"
        })

# Final dataframe
recs_same_genre_dissimilar = pd.DataFrame(rows)

In [281]:
rows = []
for subj in analysis_df["subject"].unique():
    
    # Subject's preferred genre
    preferred_genre = analysis_df.loc[
        analysis_df["subject"] == subj, "preferred_genre"
    ].iloc[0]
    
    # Subject's like profile
    user_pref = analysis_df.loc[
        analysis_df["subject"] == subj, like_cols
    ].iloc[0]
    
    # Songs in any genre except  the preferred one
    songs_diff_genre = df_songs[df_songs["genre_allmusic"] != preferred_genre].copy()
    
    # Remove songs already rated by this subject
    rated_songs = set(
        analysis_df.loc[analysis_df["subject"] == subj, "song_id"].tolist()
    )
    songs_diff_genre = songs_diff_genre[
        ~songs_diff_genre["id"].isin(rated_songs)
    ].copy()
    
    # Compute distance to subject’s preference
    diffs = songs_diff_genre[feat_cols].values - user_pref.values
    songs_diff_genre["distance"] = np.linalg.norm(diffs, axis=1)
    
    # Take N least similar songs (largest distance)
    most_dissimilar = songs_diff_genre.sort_values("distance", ascending=False).head(N)
    
    # Keep only requested columns
    for _, row in most_dissimilar.iterrows():
        rows.append({
            "subject": subj,
            "genre_allmusic": row["genre_allmusic"],
            "trackname": row["trackname"],
            "bucket": "diff_genre_dissimilar"
        })

# Final dataframe
recs_diff_genre_dissimilar = pd.DataFrame(rows)

Now that we have all these, we put them together in a single dataframe

In [282]:
all_recs = pd.concat(
    [
        recs_same_genre_similar,
        recs_same_genre_dissimilar,
        recs_diff_genre_similar,
        recs_diff_genre_dissimilar,
    ],
    ignore_index=True
)
#sort all_recs
all_recs = all_recs.sort_values(
    by=["subject", "bucket"],
    key=lambda x: x.map({
        "same_genre_similar": 0,
        "same_genre_dissimilar": 1,
        "diff_genre_similar": 2,
        "diff_genre_dissimilar": 3
    }) if x.name == "bucket" else x
).reset_index(drop=True)
all_recs.head()

,subject,genre_allmusic,trackname,bucket
0,S105,pop-rock,Runaway Train,same_genre_similar
1,S105,pop-rock,Nutshell,same_genre_similar
2,S105,pop-rock,I'm a Slave 4 U,same_genre_dissimilar
3,S105,pop-rock,Message In A Bottle,same_genre_dissimilar
4,S105,country,Three Wooden Crosses,diff_genre_similar


In [283]:
# Merge to add email per subject
all_recs_with_email = all_recs.merge(contact_group2, on="subject", how="left")

# Remove genre column
all_recs_with_email = all_recs_with_email.drop(columns=["genre_allmusic"])

all_recs_with_email


,subject,trackname,bucket,email
0,S105,Runaway Train,same_genre_similar,kittytail@hotmail.co.uk
1,S105,Nutshell,same_genre_similar,kittytail@hotmail.co.uk
2,S105,I'm a Slave 4 U,same_genre_dissimilar,kittytail@hotmail.co.uk
3,S105,Message In A Bottle,same_genre_dissimilar,kittytail@hotmail.co.uk
4,S105,Three Wooden Crosses,diff_genre_similar,kittytail@hotmail.co.uk
5,S105,Trigger,diff_genre_similar,kittytail@hotmail.co.uk
6,S105,"Cello Concerto in E Minor, Op.85 (1997 - Remas...",diff_genre_dissimilar,kittytail@hotmail.co.uk
7,S105,"Water Music Suite No. 1 In F, HWV 348: 2. Adag...",diff_genre_dissimilar,kittytail@hotmail.co.uk
8,S106,"More Than A Woman - From ""Saturday Night Fever...",same_genre_similar,harryhall3698@gmail.com
9,S106,Nutshell,same_genre_similar,harryhall3698@gmail.com
